Lab Group: Lianhe Chu, Jie Zhong

3. Random indexing with permutations
    1. You have got the code to train word embedding using Random indexing.
    2. The code also estimates the performance on TOEFL task
    3. The current code works adequately only when size of window is 2. What
    should be changed in order to overcome this issue?
    4. Get the performance of the model for three different dimensionalities. The
    choice of dimensionalities is on your own but use different values (e.g.,
    1000, 4000, 10000).
    5. If your computational resources allow run simulations several times (e.g.
    5) for each dimensionality
    6. Report the accuracy on TOEFL for all simulations
    7. Elaborate how accuracy changes with the dimensionality

In [1]:
import numpy as np
import text_functions as tf
import nltk

# Function for dimension

In [3]:
def dimension_study(dimensionnumber):
    a = 0.0  # accuracy of the encodings
    # Number of times to run the code
    num_runs = 5

    for run in range(num_runs):
        print(f"\nRun {run + 1}:")


        threshold = 15000 # Frequency threshold in the corpus ??
        dimension = dimensionnumber # Dimensionality for high-dimensional vectors
        lemmatizer = nltk.WordNetLemmatizer()  # create an instance of lemmatizer
        ones_number = 2 # number of nonzero elements in randomly generated high-dimensional vectors
        window_size = 2 #number of neighboring words to consider both back and forth. 
        #In other words number of words before/after current word
        zero_vector = np.zeros(dimension)
        test_name = "new_toefl.txt" # file with TOEFL dataset
        data_file_name = "lemmatized.text" # file with the text corpus

        amount_dictionary = {}

        # Count how many times each word appears in the corpus
        text_file = open(data_file_name, "r")
        for line in text_file:
            if line != "\n":
                words = line.split()
                for word in words:
                    if amount_dictionary.get(word) is None:
                        amount_dictionary[word] = 1
                    else:
                        amount_dictionary[word] += 1
        text_file.close()

        dictionary = {} #vocabulary and corresponing random high-dimensional vectors
        word_space = {} #embedings

        #Create a dictionary with the assigned random high-dimensional vectors
        text_file = open(data_file_name, "r")
        for line in text_file: #read line in the file
            words = line.split() # extract words from the line
            for word in words:  # for each word
                if dictionary.get(word) is None: # If the word was not yed added to the vocabulary
                    if amount_dictionary[word] < threshold:
                        dictionary[word] = tf.get_random_word_vector(dimension, ones_number) # assign a  
                    else:
                        dictionary[word] = np.zeros(dimension) # frequent words are assigned with empty vectors. In a way they will not contribute to the word embedding

        text_file.close()


        #Note that in order to save time we only create embeddings for the words needed in the TOEFL task

            #Find all unique words amongst TOEFL tasks and initialize their embeddings to zeros    
        number_of_tests = 0
        text_file = open(test_name, "r") #open TOEFL tasks
        for line in text_file:
                words = line.split()
                words = [lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word, 'v'), 'n'), 'a') for word in
                        words] # lemmatize words in the current test
                word_space[words[0]] = np.zeros(dimension)
                word_space[words[1]] = np.zeros(dimension)
                word_space[words[2]] = np.zeros(dimension)
                word_space[words[3]] = np.zeros(dimension)
                word_space[words[4]] = np.zeros(dimension)
                number_of_tests += 1
        text_file.close()

        # Processing the text to build the embeddings 
        text_file = open(data_file_name, "r")
        lines = [[],[],[],[]] # neighboring lines
        i = 2
        while i < 4:
                line = "\n"
                while line == "\n":
                    line = text_file.readline()
                lines[i] = line.split()
                i += 1

        line = text_file.readline()
        while line != "":
                if line != "\n":
                    lines.append(line.split())
                    words = lines[2]
                    length = len(words)
                    i = 0
                    while i < length:
                        if not (word_space.get(words[i]) is None):
                            k = 1
                            word_space_vector = word_space[words[i]]
                            while (i - k >= 0) and (k <= window_size): #process left neighbors of the focus word
                                word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[words[i - k]], -1))         
                                k += 1
                            # Handle different situations if there was not enough neighbors on the left in the current line    
                            if k <= window_size and (len(lines[1])>0): 
                                if len(lines[1]) < 2:
                                    if k != 1: #if one word on the left was already added
                                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[1][0]], -1)) #update word embedding
                                    else:
                                        word_space[words[i]] = np.add(word_space[words[i]],
                                                                    np.roll(dictionary[lines[1][0]], -1)) #update word embedding
                                        word_space[words[i]] = np.add(word_space[words[i]],
                                                                    np.roll(dictionary[lines[0][len(lines[0]) - 1]], -1)) #update word embedding
                                else:
                                    if k != 1:
                                        word_space[words[i]] = np.add(word_space[words[i]],
                                                                    np.roll(dictionary[lines[1][len(lines[1]) - 1]], -1)) #update word embedding
                                    else:
                                        word_space[words[i]] = np.add(word_space[words[i]],
                                                                    np.roll(dictionary[lines[1][len(lines[1]) - 1]], -1)) #update word embedding
                                        word_space[words[i]] = np.add(word_space[words[i]],
                                                                    np.roll(dictionary[lines[1][len(lines[1]) - 2]], -1)) #update word embedding

                            k = 1
                            while (i + k < length) and (k <= window_size): #process right neighbors of the focus word
                                word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[words[i + k]], 1)) #update word embedding
                                k += 1
                            if k <= window_size:
                                if len(lines[3]) < 2:
                                    if k != 1:
                                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                    else:
                                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[4][0]], 1)) #update word embedding
                                else:
                                    if k != 1:
                                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                    else:
                                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                        word_space[words[i]] = np.add(word_space[words[i]],
                                                                np.roll(dictionary[lines[3][1]], 1))

                        i += 1
                    lines.pop(0)
                line = text_file.readline()

        # Testing of the embeddings on TOEFL

        i = 0
        text_file = open(test_name, 'r')
        right_answers = 0.0  # variable for correct answers
        number_skipped_tests = 0.0  # some tests could be skipped if there are no corresponding words in the vocabulary extracted from the training corpus
        while i < number_of_tests:
            line = text_file.readline()  # read line in the file
            words = line.split()  # extract words from the line
            words = [lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word, 'v'), 'n'), 'a') for word in
                    words]  # lemmatize words in the current test
            try:

                if not (amount_dictionary.get(words[0]) is None):  # check if there word in the corpus for the query word
                    k = 1
                    while k < 5:
                        if np.array_equal(word_space[words[k]], zero_vector):  # if no representation was learned assign a random vector
                            word_space[words[k]] = np.random.randn(dimension)
                        k += 1
                    right_answers += tf.get_answer_mod(
                        [word_space[words[0]], word_space[words[1]], word_space[words[2]],
                        word_space[words[3]], word_space[words[4]]])  # check if the word is predicted right
            except KeyError:  # if there is no representation for the query vector then skip
                number_skipped_tests += 1
                print("skipped test: " + str(i) + "; Line: " + str(words))
            except IndexError:
                print(i)
                print(line)
                print(words)
                break
            i += 1
        text_file.close()
        a += 100 * right_answers / number_of_tests
        print(str(dimension) + " Dimension percentage of correct answers: " + str(100 * right_answers / number_of_tests) + "%")
    print(" average accuarcy: " + str(a/num_runs) + "%")

# Loop over function

In [4]:
dimensionlist = [1000, 4000, 10000]
for dimnr in dimensionlist:
    dimension_study(dimnr)


Run 1:
1000 Dimension percentage of correct answers: 73.75%

Run 2:
1000 Dimension percentage of correct answers: 67.5%

Run 3:
1000 Dimension percentage of correct answers: 72.5%

Run 4:
1000 Dimension percentage of correct answers: 70.0%

Run 5:
1000 Dimension percentage of correct answers: 68.75%
 average accuarcy: 70.5%

Run 1:
4000 Dimension percentage of correct answers: 72.5%

Run 2:
4000 Dimension percentage of correct answers: 71.25%

Run 3:
4000 Dimension percentage of correct answers: 72.5%

Run 4:
4000 Dimension percentage of correct answers: 72.5%

Run 5:
4000 Dimension percentage of correct answers: 71.25%
 average accuarcy: 72.0%

Run 1:
10000 Dimension percentage of correct answers: 75.0%

Run 2:
10000 Dimension percentage of correct answers: 75.0%

Run 3:
10000 Dimension percentage of correct answers: 70.0%

Run 4:
10000 Dimension percentage of correct answers: 75.0%

Run 5:
10000 Dimension percentage of correct answers: 73.75%
 average accuarcy: 73.75%


# Report the average accuracy on TOEFL for all simulations: 
* 1000 dimension: 70.5%
* 4000 dimension: 72.0%%
* 10000 dimension: 73.75%

# Elaborate how accuracy changes with the dimensionality:

* Observation: the accuary increases with the increased dimensionality with the above study